In [ ]:
!pip install ortools

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import time

In [ ]:
# Problem data initialization
data = {}
data["distance_matrix"] = [
      [0, 12, 6, 9], # 'S' distances
      [12, 0, 7, 6], # 'A' distances
      [6, 7, 0, 6],  # 'B' distances
      [9, 6, 6, 0]   # 'C' distances
]
data["num_vehicles"] = 1 # Represents a single customer (TSP)
data["depot"] = 0        # Index of the start/end node ('S')

# Routing model creation
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)
routing = pywrapcp.RoutingModel(manager)

# Callback: returns distance between two nodes
def distance_callback(from_index, to_index):
    # Convert from routing variable Index to distance matrix NodeIndex
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

# Register the distance callback and set cost evaluator
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Set search strategy: greedy initialization using PATH_CHEAPEST_ARC
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)
if solution:
    total_distance = solution.ObjectiveValue()
    index = routing.Start(0)
    optimal_path = []
    while not routing.IsEnd(index):
        optimal_path.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    print(f"Near-optimal shopping path: {optimal_path}")

Near-optimal shopping path: [0, 2, 1, 3]
